In [ ]:
import sys
from pathlib import Path

import pandas as pd

In [ ]:
TOP_DIR = Path('../../../').resolve()
if str(TOP_DIR) not in sys.path: sys.path.append(str(TOP_DIR))

In [ ]:
from lib.util.geo import postcode_formatter, postcode_to_ward_code

In [ ]:
data = pd.read_csv(
    '../../../working/metrics/partnerships/all.csv',
    parse_dates=['Start date', 'End date'],
    usecols=[
        # 'Partner Lead Contact',
        'Partner Organisation',
        'List of countries involved',
        'End date',
        # 'Baseline survey completed by partner',
        'Start date',
        'Partner Venue Postcode',
        # 'Notes',
        'Partnership Status',
        'Project Artform',
        # 'Project Description',
        'Project Name',
        # 'Offered SAIL training',
        'Project Output',
        # 'L23 economic contribution',
        'International element to project?',
        'Online/in person/both',
        'Number of countries involved',
        'Season',
        # 'Partner Venue Name',
        # 'Contract signed by partner',
        'Duration of activity/event',
        # 'LEEDS 2023 Assignee.id',
        # 'LEEDS 2023 Assignee.email',
        # 'LEEDS 2023 Assignee.name',
        # 'Last Modified By.id',
        # 'Last Modified By.email',
        # 'Last Modified By.name',
        # 'Activity log completed by partner',
        # 'Final survey completed by partner',
        # 'Number of unique countries',
        'Contains heritage, young people',
        # 'Total budget',
        'Digital audiences',
        'Audience numbers',
        'number of participants',
        'Total number of events'
    ]
)

In [ ]:
data.columns.to_list()

In [ ]:
data.columns = data.columns.str.strip().str.lower().str.replace(r'[^a-z0-9]+', '_', regex=True)

In [ ]:
data['partner_venue_ward_code'] = data['partner_venue_postcode'].pipe(postcode_formatter).map(postcode_to_ward_code)

In [ ]:
data['list_of_countries_involved'] = (
  data['list_of_countries_involved']
    .str.strip()
    .str.replace('[\.]', '', regex=True)
    .str.replace(r'[\n]+|\s*,\s*', '|', regex=True)
    .str.replace('The Netherlands','Netherlands')
    .str.replace('Trinidad & Tobago','Trinidad and Tobago')
    .str.strip('|')
    .str.split('|')
)


In [ ]:
data['number_of_countries_involved'] = data.loc[data.list_of_countries_involved.notna(), 'list_of_countries_involved'].map(len)
data['number_of_countries_involved'].fillna(0, inplace=True)

In [ ]:
OUTPUT_DIR = TOP_DIR.joinpath('data/metrics/partnerships')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
data.to_csv(
  OUTPUT_DIR.joinpath('partnerships.csv'),
  index=False
)